In [1]:
# Get ready
import backoffice
import ray
import requests
backend_config = {'num_cpus': 0.5}

In [2]:
backend_server = backoffice.start_backend_server(config=backend_config)
print(ray.available_resources())
print(ray.cluster_resources())

{'object_store_memory': 93.0, 'CPU': 4.0, 'memory': 270.0, 'node:173.249.37.119': 0.98}
{'CPU': 6.0, 'node:173.249.37.119': 1.0, 'object_store_memory': 93.0, 'memory': 270.0}


In [3]:
# Show Policies
# Todo: Refactor this code
backend_server.list_backends()


{}

In [4]:
# Show Endpoints and associated policies
backend_server.list_endpoints()

{}

In [5]:
# Show policies
backoffice.show_policies()

,trainer_id,trainer_name,cloud_provider,model_name,sim_config_id,policy_id,run_id,time_start,simulations,duration,mean,std,deployed
0,1,lairning,,gas_station,1,1,1,2021-04-15 21:39:56.714415,50,112.259772,792.423584,101.483188,False
1,1,lairning,,gas_station,1,2,2,2021-04-15 21:41:49.089736,50,115.822213,779.720210,109.804170,False
2,1,lairning,,gas_station,2,3,3,2021-04-15 21:43:45.173862,50,131.719333,776.709057,95.506141,False
3,1,lairning,,gas_station,2,4,4,2021-04-15 21:45:57.089685,50,124.567125,788.258564,83.861962,False
0,2,lairning,aws,gas_station,2,1,1,2021-04-16 16:05:56.964909,50,15.537458,312.263433,72.027725,False


In [6]:
#Deploy a policy, create an endpoint and associate the policiy with the endpoint
backoffice.deploy_endpoint_policy(backend_server, trainer_id=2, policy_id=1, endpoint_name ="light_manager")
print(ray.available_resources())

# Policy 'trainer2_policy1' Deployed
{'node:173.249.37.119': 0.97, 'object_store_memory': 93.0, 'memory': 270.0, 'CPU': 3.5}


In [32]:
backend_server.update_backend_config('trainer3_policy1',config_options={'num_replicas':2})
print(ray.available_resources())

{'memory': 270.0, 'CPU': 3.0, 'object_store_memory': 93.0, 'node:173.249.37.119': 0.98}


In [ ]:
# Test the endpoint by executing one simulation
traffic_sim = backoffice.get_simulator(trainer_id=2, policy_id=1)
observation = traffic_sim.reset()
total_reward = 0
done = False
while not done:
    response = requests.get("http://localhost:8000/light_manager", json={"observation": observation})
    action = response.json()['action']
    observation, reward, done, _ = traffic_sim.step(action)
    total_reward += reward

print(total_reward)


In [33]:
#Deploy two policies
backoffice.deploy_policy(backend_server, trainer_id=1, policy_id=1)
backoffice.deploy_policy(backend_server, trainer_id=1, policy_id=4)
print(ray.available_resources())

# Policy 'trainer1_policy1' Deployed
# Policy 'trainer1_policy4' Deployed
{'node:173.249.37.119': 0.98, 'memory': 270.0, 'object_store_memory': 93.0, 'CPU': 2.0}


In [34]:
#Create a endpoint to serve the policies
backoffice.add_endpoint(backend_server, policy_name='trainer1_policy1', endpoint_name ="truck_dispatcher")
backend_server.set_traffic("truck_dispatcher", {'trainer1_policy1':0.5, 'trainer1_policy4':0.5})

In [44]:
# Test the endpoint by executing one simulation
import requests

gas_sim = backoffice.get_simulator(trainer_id=1, policy_id=1)
observation = gas_sim.reset()
total_reward = 0
done = False
while not done:
    response = requests.get("http://localhost:8000/truck_dispatcher", json={"observation": observation})
    action = response.json()['action']
    observation, reward, done, _ = gas_sim.step(action)
    total_reward += reward

print(total_reward)


616.8400159989324


In [8]:
#Remove first end point and it's associated policiy
backend_server.delete_endpoint("light_manager")
backoffice.undeploy_policy(backend_server, policy_name='trainer2_policy1')


In [46]:
#Remove the second end point and it's associated policies
backend_server.delete_endpoint("truck_dispatcher")
backoffice.undeploy_policy(backend_server, policy_name='trainer1_policy1')
backoffice.undeploy_policy(backend_server, policy_name='trainer1_policy4')